In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Assignments/spam.csv', encoding='ISO-8859-1')

In [3]:
df

v1                                                 v2 Unnamed: 2  \
0      ham  Go until jurong point, crazy.. Available only ...        NaN   
1      ham                      Ok lar... Joking wif u oni...        NaN   
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3      ham  U dun say so early hor... U c already then say...        NaN   
4      ham  Nah I don't think he goes to usf, he lives aro...        NaN   
...    ...                                                ...        ...   
5567  spam  This is the 2nd time we have tried 2 contact u...        NaN   
5568   ham              Will Ì_ b going to esplanade fr home?        NaN   
5569   ham  Pity, * was in mood for that. So...any other s...        NaN   
5570   ham  The guy did some bitching but I acted like i'd...        NaN   
5571   ham                         Rofl. Its true to its name        NaN   

     Unnamed: 3 Unnamed: 4  
0           NaN        NaN  
1           NaN        NaN  
2           NaN        NaN  
3           NaN        NaN  
4           NaN        NaN  
...         ...        ...  
5567        NaN        NaN  
5568        NaN        NaN  
5569        NaN        NaN  
5570        NaN        NaN  
5571        NaN        NaN  

[5572 rows x 5 columns]

In [4]:
df = df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])

In [5]:
df

v1                                                 v2
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...
...    ...                                                ...
5567  spam  This is the 2nd time we have tried 2 contact u...
5568   ham              Will Ì_ b going to esplanade fr home?
5569   ham  Pity, * was in mood for that. So...any other s...
5570   ham  The guy did some bitching but I acted like i'd...
5571   ham                         Rofl. Its true to its name

[5572 rows x 2 columns]

In [6]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [7]:
# Function to clean text using spaCy
from typing import List
def spacy_clean_text(text: str) -> List[str]:
    # Process the text using spaCy
    doc = nlp(text)

    # Filter tokens: keep only alphabetic words, remove stop words and punctuations
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]

    # Join the tokens back into a cleaned string
    return ' '.join(tokens)

In [8]:
df.columns = ['label', 'message']

In [9]:
df['tokens_message'] = df['message'].apply(spacy_clean_text)

In [10]:
df.head()

label                                            message  \
0   ham  Go until jurong point, crazy.. Available only ...   
1   ham                      Ok lar... Joking wif u oni...   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...   
3   ham  U dun say so early hor... U c already then say...   
4   ham  Nah I don't think he goes to usf, he lives aro...   

                                      tokens_message  
0  jurong point crazy available bugis n great wor...  
1                              ok lar joke wif u oni  
2  free entry wkly comp win FA Cup final tkts tex...  
3                                u dun early hor u c  
4                              nah think go usf live

In [11]:
df = df.drop(columns=['message'])

In [12]:
df.columns = ['Label', 'Message']

In [13]:
df.head()

Label                                            Message
0   ham  jurong point crazy available bugis n great wor...
1   ham                              ok lar joke wif u oni
2  spam  free entry wkly comp win FA Cup final tkts tex...
3   ham                                u dun early hor u c
4   ham                              nah think go usf live

**Pipeline 1: Sparse Embeddings (TF-IDF):**

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Subset of dataset
large_df = df.sample(frac=0.4, random_state=40)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(large_df['Message'], large_df['Label'], test_size=0.2, random_state=40)

# TF-IDF Vectorizer
tfidf = TfidfVectorizer(min_df=5, max_df=0.9, ngram_range=(1,2), max_features=1000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
# Model Training with Class Weights
model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(X_train_tfidf, y_train)

# Predictions and Evaluation
y_pred = model.predict(X_test_tfidf)
f1_tfidf = f1_score(y_test, y_pred, average='macro')

print(f"F1 Score for TF-IDF only with class weights: {f1_tfidf}")

F1 Score for TF-IDF only with class weights: 0.9549767817484353
